## Independent Multiple Time Series Forecasting: 
Forecast multiple time series with a single model.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries

from utils import *

import warnings
warnings.filterwarnings("ignore")

In [3]:
np.random.seed(2024)

#### Load data

In [4]:
tukey = pd.read_pickle('data/tukey_estimates.pkl')
excess_returns = pd.read_pickle('data/exc_returns.pkl')
index_returns = pd.read_pickle('data/index_returns.pkl')
stocks_returns = pd.read_pickle('data/stocks_returns.pkl')

In [11]:
tukey = tukey.reset_index().rename(columns={'index':'Date'}).set_index('Date')
tukey.head()

,VOW3 GY Equity,ENGI FP Equity,BAYN GY Equity,BNP FP Equity,PHIA NA Equity,ISP IM Equity,SAN FP Equity,MC FP Equity,ENEL IM Equity,CS FP Equity,...,OR FP Equity,BBVA SQ Equity,ASML NA Equity,SAP GY Equity,INGA NA Equity,ENI IM Equity,EOAN GY Equity,DBK GY Equity,ABI BB Equity,DTE GY Equity
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-07,153.939608,15.863052,70.854575,44.436658,19.238627,1.380126,72.813778,127.405070,3.231326,13.850115,...,105.634999,7.310501,49.852665,61.648194,7.369072,18.959361,12.610719,29.582595,67.114440,8.771007
2013-01-08,152.295129,15.864724,70.827743,44.922120,19.152028,1.390526,73.053974,126.747672,3.202159,13.846611,...,104.938642,7.356010,49.264694,61.258046,7.436251,18.885706,12.580650,30.070623,66.745362,8.812163
2013-01-09,150.639015,15.827395,71.009542,45.323106,19.125098,1.405432,73.282788,126.031903,3.195986,13.749587,...,104.897190,7.447657,48.849551,61.011026,7.526238,18.937751,12.583433,30.564229,66.302363,8.861539
2013-01-10,149.452580,15.727265,70.822367,45.607353,19.161744,1.428110,73.181097,124.892381,3.190310,13.623088,...,104.390636,7.557309,48.616030,60.806893,7.616594,18.997714,12.550908,31.038729,65.602069,8.973196
2013-01-11,149.469007,15.647943,71.089809,45.684501,19.391704,1.453998,73.397973,124.341556,3.210371,13.641131,...,104.406391,7.695797,48.659719,60.888182,7.680333,19.172979,12.552136,31.371286,65.322242,9.052104
